In [1]:
import lammps
import pymc3 as pm
import arviz as az
import numpy as np
import glob
from copy import deepcopy
import matplotlib.pyplot as plt

from lammps_data import LammpsData
from pymatgen.io.vasp import Poscar

In [2]:
def structs_from_poscars(params):
    """
    Reads in POSCARs, creates equivalent lammps inputs.
    
    Args:
        params (dict(dict)): contains core_shell (dict (bool)), charges (dict), and masses (dict).
                             charges contains the charges for each atom type. Key = Atom label(str),
                             value = charge(float)/sub_dict(dict). If atom is core-shell a sub dictionary
                             will be the value, where sub_key = 'core' or 'shell' (float) and sub_value
                             the charge (float). Same format/types uses for masses as in charges.

    Returns:
        structures (list(str)): A list of lammps imput file names generated from the POSCARs.
        structures_data (list(obj)): A list of lammps objects generated from the POSCARs.
    """
    structures = []
    structures_data = []
    for i, pos in enumerate(glob.glob('poscars/POSCAR*')):
        poscar = Poscar.from_file(pos)
        struct_data = LammpsData.from_structure(poscar.structure, params)
        structures_data.append(struct_data)
        lammps_file = 'lammps/coords{}.lmp'.format(i+1)
        with open( lammps_file, 'w' ) as f:
            f.write(struct_data.input_string())  
        structures.append(lammps_file)
        
    return structures, structures_data

def type_shell(core_shell):
    """
    Determines which atom_type index relate to the shells on core-shell atoms
    
    Args:
        core_shell (dict): A dictionary of booleans stating if any atoms are core-shell.
    Returns:
        shell_nums (list): A list of atom_type index values for any shells present on core-shell atoms.
    """
    shell_nums = []
    x = 0
    for i in core_shell.values():
        x +=1
        if i:
            x+=1
            shell_nums.append(x)
    return shell_nums

In [3]:
def init(structure_data, structure, cs_springs=None):
    """
    Initialises the system from the structure (read in from lammps input) and non-changing parameters.
    
    Args:
        structure (str): Name of lammps input file.
        cs_springs (dict): A dictionary of the bond_coeff values. Key = atom label and
                           values = K (energy/distance^2) and r0 (equilibrium bond distance) (list).
                           Default = None.

    Returns:
        lmp (Lammps): Lammps system with structure and specified commands implemented.
    """
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data {}'.format(structure))

    lmp.command('group cores type {}'.format(structure_data.type_core()))
    lmp.command('group shells type {}'.format(structure_data.type_shell()))

    if cs_springs:
        lmp.command('pair_style buck/coul/long/cs 10.0')
        lmp.command('pair_coeff * * 0 1 0')

        lmp.command('bond_style harmonic')
        for i, spring in enumerate(cs_springs):
            lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                     cs_springs[spring][0],
                                                     cs_springs[spring][1]))
    else:
        lmp.command('pair_style buck/coul/long 10.0')
        lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('kspace_style ewald 1e-6')

    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [4]:
def dex(element):
    """
    Finds atom_index number for given element, where core-shell returns shell index.

    Args:
        element (str): element symbol

    Returns:
        atom_index (int): index of atom relating to element. In the case of core-shell atoms
                          this will return the shell index.
    """
    element_list = [e.label for e in structure_data.atom_types]
    for e in element_list:
        if element in e and 'core' not in e:
            atom_index = element_list.index(e)+1
    return atom_index

def update_potentials(**kwargs):
    """
    Unpdates the potentials set by pymc3 into the dictionary for the fitting process.

    Args:
        **kwargs: The parameters to be updated in the fitting process as set with pm.Model.

    Returns:
        None.
    """
    bpp.update(kwargs)
    
def set_potentials(instance):
    """
    Sets the potential for the sepecified Lammps system (changes for each iteration of the potential fit).

    Args:
        instance (Lammps): Lammps system with structure and specified commands implemented.

    Returns:
        None
    """
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))


In [5]:
def simfunc(**kwargs):
    """
    Runs a minimization and run for each instance and returns the forces.

    Args:
        **kwargs: Contain data for type of fitting and to what parameters as set with pm.Model.

    Returns:
        out (np.array): x,y,z forces on each atom associated with each given instance.
    """
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([len(index), 3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.command('fix 1 cores setforce 0.0 0.0 0.0')
            instance.command('minimize 1e-25 1e-25 5000 10000')
            instance.command('unfix 1')
            instance.run(0)
            out[:,:,instances.index(instance)] = np.array([value for i, value in enumerate(instance.system.forces) if i in index])
            
    else: out = np.ones([len(index),3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
    
    return out

In [6]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 15.0,
                          'shell': 1.0} }

poscar = Poscar.from_file('poscars/POSCAR1')
my_structure = poscar.structure
structure_data = LammpsData.from_structure(my_structure, params)
lammps_file = 'lammps/coords1.lmp'
with open( lammps_file, 'w' ) as f:
    f.write(structure_data.input_string())

pairs = [('Li','O'), ('Ni','O'), ('O','O')]

#Must be in same order given in charges dictionary
cs_springs = {'O' : [65.0, 0.0]} #Set to None if not using any core-shells

bpp_def = {'Li_O_A'   : 632.1018,
           'Li_O_rho' : 0.2906,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1582.5000,
           'Ni_O_rho' : 0.2882,
           'Ni_O_C'   : 0.000,
           'O_O_A'    : 22764.3000,
           'O_O_rho'  : 0.1490,
           'O_O_C'    : 21.7}

bpp = deepcopy(bpp_def)


 # 3. Load a POSCAR or list of POSCARs using pymatgen and appends to a list of structures
structures, structures_data = structs_from_poscars(params)


shells = type_shell(params['core_shell'])

Found elements: ['Li', 'Ni', 'O']
Found elements: ['Li', 'Ni', 'O']


In [7]:
instances = [init(structures_data[i], structures[i], cs_springs) for i,structure in enumerate(structures)]
index = [ i for i, atype in enumerate(instances[0].system.types) if atype not in shells]

In [8]:
expected = np.zeros([len(index),3, len(instances)])

In [ ]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  50)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
#     Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.01)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=1000)
#     trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1000, dist_func="sum_of_squared_distance"), draws=1000)

Sample initial stage: ...
/Users/lmm82/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pymc3/step_methods/smc.py:176: UserWarning: Warning: SMC-ABC methods are experimental step methods and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)
Stage: 0 Beta: 0.003 Steps: 25


In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)
plt.savefig('coreshell_LiNiO2_trace3.png',dpi=500, bbox_inches = "tight")

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')
plt.savefig('coreshell_LiNiO2_mode3.png',dpi=500, bbox_inches = "tight")

In [ ]:
# fig, axes = az.plot_forest(trace,
#                            kind='ridgeplot',
#                            var_names=['Li_O_A','Ni_O_A', 'O_O_A'],
#                            combined=True,
#                            ridgeplot_overlap=10,
#                            colors='white',
#                            figsize=(9, 7))

In [ ]:
# pm.summary(trace)

In [ ]:
# def setup_parameters():
#     """
#     Contains and returns the setup information.
    
#     Args:
#         None.

#     Returns:
#         core_shell (dict): A dictionary of booleans defining which elements are core-shell species.
#                            Key = atom label (str) and value = True or False (bool)
#         charges (dict): A dictionary of charges for each atom type. Key = Atom label(str),
#                         value = either the charge (float) for non-core-shell species, or a sub-dictionary
#                         containing { 'core': core_charge (float), 'shell': shell_charge (float) }.
#         structures (list): A list of lammps input file names generated from the POSCARs.
#         elements (list): A list of the elements in the structure, with core and shell entered separately.
#         pairs (list): A list of interacting atom pairs used for defining the pair_coeff.
#         cs_springs (dict): A dictionary of the bond_coeff values. Key = atom label and
#                            values = K (energy/distance^2) and r0 (equilibrium bond distance) (list).
#         bpp_def (dict): A dictionary of the default/starting values for the buckingham potentials.
#                         Key = atom1_atom2_parameter (str) and value = parameter (float).
#         bpp (dict): Same as bpp_def however the values will change during the fitting process.
#                     Key = atom1_atom2_parameter (str) and value = parameter (float).
#     """
    
#     return  structures, structures_data, params, pairs, cs_springs, bpp_def, bpp

# structures, structures_data, params, pairs, cs_springs, bpp_def, bpp = setup_parameters()